<a href="https://colab.research.google.com/github/PragatheeswaranR/Analytics-Vidhya-/blob/master/Janata%20hack%20Computer%20vision%20Hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# We first load the necessary libraries, the dataset
import tensorflow as tf
from keras import callbacks
from keras import optimizers
from keras import Model
from keras.layers import Dropout, Flatten, Dense
from keras.optimizers import Adam
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.image as mpimg
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D,AveragePooling2D,BatchNormalization,GlobalMaxPooling2D,GlobalAveragePooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tqdm import tqdm
from keras.applications import VGG16
from keras.datasets import cifar10
from keras.utils import to_categorical
import numpy as np

Using TensorFlow backend.


In [2]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
download = drive.CreateFile({'id': '1GkNrYeYWqiWrHsuaVBgKNiP0MxNCvIbu'})
download.GetContentFile('Train_traffic.zip')
!unzip Train_traffic.zip



In [4]:
train = pd.read_csv('train.csv')
train.head()

,image_names,emergency_or_not
0,1503.jpg,0
1,1420.jpg,0
2,1764.jpg,0
3,1356.jpg,0
4,1117.jpg,0


In [5]:
# Converting images to array
train_image = []
for i in tqdm(range(train.shape[0])):
  img = image.load_img('images/'+train['image_names'][i],grayscale=False)
  img = image.img_to_array(img)
  img = img/255
  train_image.append(img)
X = np.array(train_image)

100%|██████████| 1646/1646 [00:02<00:00, 669.14it/s]


In [6]:
#Converting target array to one hot vector
y=train['emergency_or_not'].values
y = to_categorical(y)

In [7]:
# We build the base model with pretrained weights
input_shape = (224, 224, 3)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
base_model.summary()

58892288/58889256 [==============================] - 3s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
______________________________________________________________

In [8]:
# We freeze every layer in our base model so that they do not train, we want that our feature extractor stays as before --> transfer learning
for layer in base_model.layers: 
  layer.trainable = False
  print('Layer ' + layer.name + ' frozen.')
# We take the last layer of our the model and add it to our classifier
last = base_model.layers[-1].output
x = Flatten()(last)
x = Dense(1000, activation='relu', name='fc1')(x)
x = Dropout(0.3)(x)
x = Dense(2, activation='sigmoid', name='predictions')(x)
model = Model(base_model.input, x)
# We compile the model
model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()



Layer input_1 frozen.
Layer block1_conv1 frozen.
Layer block1_conv2 frozen.
Layer block1_pool frozen.
Layer block2_conv1 frozen.
Layer block2_conv2 frozen.
Layer block2_pool frozen.
Layer block3_conv1 frozen.
Layer block3_conv2 frozen.
Layer block3_conv3 frozen.
Layer block3_pool frozen.
Layer block4_conv1 frozen.
Layer block4_conv2 frozen.
Layer block4_conv3 frozen.
Layer block4_pool frozen.
Layer block5_conv1 frozen.
Layer block5_conv2 frozen.
Layer block5_conv3 frozen.
Layer block5_pool frozen.
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
__________________

In [9]:
# splitting to train validate the data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.3)

In [10]:
# We start the training
epochs = 20
batch_size = 256
# We train it
model.fit(X_train,y_train,
          batch_size=batch_size,
          validation_data=(X_test, y_test),
          epochs=epochs)

Train on 1152 samples, validate on 494 samples
Epoch 1/20
1152/1152 [==============================] - 49s 43ms/step - loss: 7.3130 - accuracy: 0.5599 - val_loss: 2.1959 - val_accuracy: 0.5435
Epoch 2/20
1152/1152 [==============================] - 9s 8ms/step - loss: 1.8157 - accuracy: 0.5104 - val_loss: 0.6042 - val_accuracy: 0.7439
Epoch 3/20
1152/1152 [==============================] - 9s 8ms/step - loss: 0.9245 - accuracy: 0.6840 - val_loss: 0.6580 - val_accuracy: 0.7014
Epoch 4/20
1152/1152 [==============================] - 9s 8ms/step - loss: 0.5617 - accuracy: 0.7413 - val_loss: 0.5821 - val_accuracy: 0.7136
Epoch 5/20
1152/1152 [==============================] - 9s 8ms/step - loss: 0.4346 - accuracy: 0.7999 - val_loss: 0.4530 - val_accuracy: 0.8107
Epoch 6/20
1152/1152 [==============================] - 9s 8ms/step - loss: 0.3804 - accuracy: 0.8372 - val_loss: 0.3556 - val_accuracy: 0.8472
Epoch 7/20
1152/1152 [==============================] - 9s 8ms/step - loss: 0.2779 - ac

In [11]:
# We evaluate the accuracy and the loss in the test set
scores = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

494/494 [==============================] - 7s 14ms/step
Test loss: 0.2127039868098039
Test accuracy: 0.9210526347160339


In [12]:
link = '1xCWn0qdgiq9WH4-otzqXhdcklr_9LqL2'
download = drive.CreateFile({'id': link})
download.GetContentFile('test_computer_vision.csv')
test_file = pd.read_csv('test_computer_vision.csv')
test_file.head()

,image_names
0,1960.jpg
1,668.jpg
2,2082.jpg
3,808.jpg
4,1907.jpg


In [13]:
test_image = []
for i in tqdm(range(test_file.shape[0])):
    img = image.load_img('images/'+test_file['image_names'][i], grayscale=False)
    img = image.img_to_array(img)
    img = img/255
    test_image.append(img)
test = np.array(test_image)

100%|██████████| 706/706 [00:00<00:00, 832.29it/s]


In [14]:
prediction = model.predict(test)

In [15]:
pred = np.argmax(prediction,axis=1)

In [16]:
download = drive.CreateFile({'id': '1myXwuDBbpNeVd0qAHCWa25mJECyctEZ0'})
download.GetContentFile('sample_submission_computer vision.csv')

In [17]:
sample = pd.read_csv('sample_submission_computer vision.csv')
sample['image_names'] = test_file['image_names']
sample['emergency_or_not'] = pred
sample.to_csv('vgc_fine_tune.csv', header=True, index=False)

In [18]:
from google.colab import files
files.download("vgc_fine_tune.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [18]:
# Your private score for this submission is : 0.9330143540669856,
# your rank  : 226